In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pickle

In [ ]:
X = pickle.load( open( "/kaggle/input/mlub-unsupervised-learning/X_train.p", "rb" ) )
y = pickle.load( open( "/kaggle/input/mlub-unsupervised-learning/y_train.p", "rb" ) )
X_unlabeled = pickle.load( open( "/kaggle/input/mlub-unsupervised-learning/X_unlabeled.p", "rb" ) )
X_test = pickle.load( open( "/kaggle/input/mlub-unsupervised-learning/X_test.p", "rb" ) )


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val= train_test_split(X, y, test_size=0.05, random_state=66)
X_unlabeled_train, X_unlabeled_val = train_test_split(X_unlabeled, test_size=0.05, random_state=66, shuffle = True)

X_unlabeled_train = X_unlabeled_train.astype('float64')
X_unlabeled_val = X_unlabeled_val.astype('float64')
X_unlabeled_train /= 255
X_unlabeled_val /= 255

X_unlabeled = X_unlabeled.astype('float64')
X_unlabeled /= 255

# normalize data
X_train = X_train.astype('float64')
X_val = X_val.astype('float64')
X_train /= 255
X_val /= 255
X_test = X_test.astype('float64')
X_test /= 255

In [ ]:
def add_noise_and_clip_data(data):
    noise = np.random.normal(loc=0.0, scale=0.1, size=data.shape)
    data = data + noise
    data = np.clip(data, 0., 1.)
    return data

X_unlabeled_noise_train = add_noise_and_clip_data(X_unlabeled_train)
X_unlabeled_noise_val = add_noise_and_clip_data(X_unlabeled_val)
X_unlabeled_noise = add_noise_and_clip_data(X_unlabeled)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

#it_train = train_datagen.flow(X_unlabeled_train, X_unlabeled_train,batch_size=128)
it_train = train_datagen.flow(X_unlabeled_noise_train, X_unlabeled_noise_train,batch_size=128)
steps = it_train.n // 128

In [ ]:
import matplotlib.pyplot as plt
x, y = next(it_train)
plt.imshow(x[0].astype('float32'));

In [ ]:
print(X_train[0].max(), X_train[0].min())

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from tensorflow.keras.layers import Flatten, Conv2D, Input, Dense, Reshape, Conv2DTranspose,\
   Activation, BatchNormalization, ReLU, Concatenate, Dropout, MaxPooling2D, UpSampling2D
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
input_img = Input(shape=(32,32,3))

In [ ]:
input_img = Input(shape=(32, 32, 3), name='ae_input')
def autoencoder(input_img):
    x = Conv2D(32, 3, activation='relu', padding='same')(input_img) #32 x 32 x 32
    x = BatchNormalization()(x)
    x = Conv2D(32, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x) #16 x 16 x 32
    x = Dropout(0.2)(x)
    
    x = Conv2D(64, 3, activation='relu', padding='same')(x) #16 x 16 x 64
    x = BatchNormalization()(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x) #8 x 8 x 64
    x = Dropout(0.3)(x)
    
    x = Conv2D(128, 3, activation='relu', padding='same')(x) #32 x 32 x 32
    x = BatchNormalization()(x)
    x = Conv2D(128, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    
    x = Conv2D(32, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    encoded = x
    
    x = Conv2D(128, 3, activation='relu', padding='same')(x) #8 x 8 x 128
    x = BatchNormalization()(x)
    x = Conv2D(128, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2,2))(x)
    
    x = Conv2D(64, 3, activation='relu', padding='same')(x) #8 x 8 x 128
    x = BatchNormalization()(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2,2))(x)
    
    x = Conv2D(32, 3, activation='relu', padding='same')(x) #8 x 8 x 128
    x = BatchNormalization()(x)
    x = Conv2D(32, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x) # 32 x 32 x 3
    return encoded,decoded

encoded, decoded = autoencoder(input_img)

ae = Model(input_img,decoded)
# encoder - utilizado para reducir la dimensión
encoder = Model(input_img, encoded)
decoder = Model(input_img, decoded)

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=15)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("ae.h5", save_best_only=True, verbose = 1)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, min_delta=0.0001)
callbacks = [early_stopping_cb,model_checkpoint_cb,reduce_lr]


In [ ]:
ae.summary()
encoder.summary()
decoder.summary()

In [ ]:
def rounded_accuracy(y_true, y_pred):
    return keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))

In [ ]:
ae.compile(loss='mean_squared_error', optimizer = "adam",metrics=[rounded_accuracy,tf.keras.metrics.RootMeanSquaredError()])
history_ae = ae.fit(it_train,steps_per_epoch=steps,epochs=30,verbose=1, validation_data = (X_unlabeled_noise_val,X_unlabeled_val),callbacks=callbacks)

#history_ae = ae.fit(X_unlabeled_noise, X_unlabeled, batch_size = 128, validation_data=(X_unlabeled_noise, X_unlabeled), epochs = 20, callbacks=callbacks)
ae.save('ae.h5')
ae.save_weights('autoencoder_1.h5')

In [ ]:
X_unlabeled_val_noised = ae.predict(X_unlabeled_noise)

In [ ]:
import matplotlib.pyplot as plt
restored_img = ae.predict(X_unlabeled_noise)
for i in range(5):
    
    plt.imshow(X_unlabeled[i])
    plt.gray()
    plt.show()
    
    plt.imshow(X_unlabeled_noise[i])
    plt.gray()
    plt.show()
    
    plt.imshow(restored_img[i])
    plt.gray()
    plt.show()
    

In [ ]:
test_data_denoised = ae.predict(X_unlabeled_noise)
idx = 4
plt.subplot(1,3,1)
plt.imshow(X_unlabeled[idx])
plt.title('original')
plt.subplot(1,3,2)
plt.imshow(X_unlabeled_noise[idx])
plt.title('noisy')
plt.subplot(1,3,3)
plt.imshow(test_data_denoised[idx])
plt.title('denoised')
plt.show()

In [ ]:
def mse(data_1, data_2):
    return np.square(np.subtract(data_1, data_2)).mean()
noisy_clean_mse = mse(X_unlabeled, X_unlabeled_val_noised)
denoised_clean_mse = mse(X_unlabeled_noise, X_unlabeled)

noisy_clean_mse, denoised_clean_mse

In [ ]:
import matplotlib.pyplot as plt
def show_image(x):
    plt.imshow(np.clip(x, 0, 1))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history_ae.history['loss'])
plt.plot(history_ae.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
def cnn(encode = None):
    x = Flatten()(encode)
    x = Dense(2048, activation='selu', kernel_initializer='he_uniform')(x)
    x = Dropout(0.5)(x)
    x = Dense(2048, activation='selu', kernel_initializer='he_uniform')(x)
    x = Dropout(0.5)(x)
    x = Dense(2048, activation='selu', kernel_initializer='he_uniform')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='selu', kernel_initializer='he_uniform')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    out = Dense(100, activation='softmax')(x)
    return out

In [ ]:
model = Model(input_img, cnn(encoded))

In [ ]:
model.summary()

In [ ]:
for l1 in model.layers[0:20]:
    print(l1)

In [ ]:
for l1,l2 in zip(model.layers[0:20],ae.layers[0:20]):
    l1.set_weights(l2.get_weights())
for layer in model.layers[0:20]:
    layer.trainable = True

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
early_stopping_cb = keras.callbacks.EarlyStopping(patience=15)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("ae.h5", save_best_only=True, verbose = 1)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2, patience=5, min_delta=0.0001)
callbacks = [model_checkpoint_cb,reduce_lr]

In [ ]:
# Tanto el SGD, como el adam modificado funciona bien

In [ ]:
from keras.optimizers import SGD, Adam, RMSprop, Adadelta
adam = tf.keras.optimizers.Adam(learning_rate = 0.00001, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)
opt = SGD(lr=0.001, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy",optimizer=adam, metrics=["accuracy"])

In [ ]:
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
it_train = datagen.flow(X_train, y_train, batch_size=64)
steps = int(X_train.shape[0] // 64)
#history = model.fit_generator(it_train,steps_per_epoch=steps,epochs=100,verbose=1,validation_data=(X_val, y_val),callbacks=callbacks, shuffle = False)
history = model.fit(X_train, y_train, batch_size=32, validation_data=(X_val, y_val),epochs=200,verbose=1,callbacks=callbacks)


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict(X_test)
y_pred = [np.argmax(y) for y in y_pred ]
df = pd.DataFrame(y_pred)
df.columns =['label']
df.index.name = 'index'
df.to_csv('y_output.csv')